In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from scipy.optimize import curve_fit
import shap
from tqdm import tqdm

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

F:\ProgramData\miniconda3\envs\nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Add features

In [2]:
# def find_grad(data_used, col_use, length=10):
    
#     def func(x, a, b):
#         y = a*x + b
#         return y
    
#     df_temp = pd.DataFrame(index=data_used.index)
#     df_temp[f'gradx_{col_use}_{length}'] = np.nan

#     for idx in data_used.index:
#         try:
#             a, b = curve_fit(func, xdata = np.arange(1,length+1), ydata = data_used.loc[idx-(length-1):idx, col_use])[0]
#             df_temp.loc[idx, f'gradx_{col_use}_{length}'] = a
#         except:
#             df_temp.loc[idx, f'gradx_{col_use}_{length}'] = np.nan

#     return df_temp

In [3]:
# for length in [60, 120]:
#     for counter in [3, 5, 10, 20, 60, 120]:
#         for icol in [f'RSI_{counter}', f'EMA_{counter}', f'SMA_{counter}']:
#             df_temp = find_grad(df, col_use=icol, length=length)
#             df = df.merge(df_temp, left_index=True, right_index=True, how='left')

In [4]:
# df = df.dropna()

In [5]:
# df.describe()

In [6]:
# df.head(-10)

In [7]:
# for icol in df.columns:
#     print(f"'{icol}',")
    

# All Data

In [2]:
active_tickers = pd.read_csv("datas//us_volume_leaders.csv")
tickers = active_tickers['Symbol'].tolist()

In [3]:
train_ratio = 0.5

df_full_train = pd.DataFrame()
df_full_test = pd.DataFrame()
df_full_l20d = pd.DataFrame()

for ticker in tqdm(tickers):
    try:
        df_used = pd.read_parquet(f"datas//ml_data//ml_feature_label_{ticker}.parquet")
        dF_used = df_used.dropna()
        train_count = int(train_ratio*len(df_used)//1)
        df_used['ticker'] = ticker
        df_full_train = pd.concat([df_full_train, df_used.loc[df_used.index[:train_count-1]]], axis=0, ignore_index=True)
        df_full_test = pd.concat([df_full_test, df_used.loc[df_used.index[train_count:-100]]], axis=0, ignore_index=True)
        df_full_l20d = pd.concat([df_full_l20d, df_used.loc[df_used.index[-100:]]], axis=0, ignore_index=True)
    except:
        print(f"skip {ticker}")

df_full_train.to_parquet("datas//ml_data2//all_train.parquet")
df_full_test.to_parquet("datas//ml_data2//all_test.parquet")
df_full_l20d.to_parquet("datas//ml_data2//all_current.parquet")

  2%|██                                                                                | 5/201 [00:00<00:24,  8.12it/s]

skip JDZG


 72%|█████████████████████████████████████████████████████████▋                      | 145/201 [01:51<01:05,  1.17s/it]

skip NNE


 82%|█████████████████████████████████████████████████████████████████▋              | 165/201 [02:15<00:46,  1.29s/it]

skip ORKT


100%|████████████████████████████████████████████████████████████████████████████████| 201/201 [03:04<00:00,  1.09it/s]


skip Downloaded from Barchart.com as of 10-20-2024 06:54am CDT


In [10]:
# df_full_l20d[['ticker','date']].groupby('ticker').max()

In [11]:
df_full_train = pd.read_parquet("datas//ml_data2//all_train.parquet")
df_full_test = pd.read_parquet("datas//ml_data2//all_test.parquet")
df_full_l20d = pd.read_parquet("datas//ml_data2//all_current.parquet")

In [12]:
df_full_l20d.dtypes

date                        datetime64[ns]
close                              float64
open                               float64
high                               float64
low                                float64
volume                             float64
RSI_3                              float64
RSI_5                              float64
RSI_10                             float64
RSI_15                             float64
RSI_20                             float64
RSI_60                             float64
RSI_120                            float64
EMA_3                              float64
EMA_5                              float64
EMA_10                             float64
EMA_15                             float64
EMA_20                             float64
EMA_60                             float64
EMA_120                            float64
SMA_3                              float64
SMA_5                              float64
SMA_10                             float64
SMA_15     

## Additional Features

In [13]:
# df_full_train.dtypes

In [14]:
# for icount in [3,5,10,15,20, 60, 120]:
#     df_full_train[f'r_dmp_adx_{icount}'] = (df_full_train[f'DMP_{icount}'] / df_full_train[f'ADX_{icount}']).copy()
#     df_full_train[f'r_dmn_adx_{icount}'] = (df_full_train[f'DMN_{icount}'] / df_full_train[f'ADX_{icount}']).copy()
    
#     df_full_test[f'r_dmp_adx_{icount}'] = (df_full_test[f'DMP_{icount}'] / df_full_test[f'ADX_{icount}']).copy()
#     df_full_test[f'r_dmn_adx_{icount}'] = (df_full_test[f'DMN_{icount}'] / df_full_test[f'ADX_{icount}']).copy()
    
#     df_full_l20d[f'r_dmp_adx_{icount}'] = (df_full_l20d[f'DMP_{icount}'] / df_full_l20d[f'ADX_{icount}']).copy()
#     df_full_l20d[f'r_dmn_adx_{icount}'] = (df_full_l20d[f'DMN_{icount}'] / df_full_l20d[f'ADX_{icount}']).copy()

In [15]:

# df_full_train[f'r_av_v5_10'] = (df_full_train[f'avg_vol5'] / df_full_train[f'avg_vol10']).copy()
# df_full_train[f'r_av_v5_20'] = (df_full_train[f'avg_vol5'] / df_full_train[f'avg_vol20']).copy()
# df_full_train[f'r_av_v5_60'] = (df_full_train[f'avg_vol5'] / df_full_train[f'avg_vol60']).copy()
# df_full_train[f'r_av_v10_20'] = (df_full_train[f'avg_vol10'] / df_full_train[f'avg_vol20']).copy()
# df_full_train[f'r_av_v10_60'] = (df_full_train[f'avg_vol10'] / df_full_train[f'avg_vol60']).copy()
# df_full_train[f'r_av_v10_120'] = (df_full_train[f'avg_vol10'] / df_full_train[f'avg_vol120']).copy()
# df_full_train[f'r_av_v20_60'] = (df_full_train[f'avg_vol20'] / df_full_train[f'avg_vol60']).copy()
# df_full_train[f'r_av_v20_120'] = (df_full_train[f'avg_vol20'] / df_full_train[f'avg_vol120']).copy()
# df_full_train[f'r_av_v60_120'] = (df_full_train[f'avg_vol60'] / df_full_train[f'avg_vol120']).copy()

# df_full_test[f'r_av_v5_10'] = (df_full_test[f'avg_vol5'] / df_full_test[f'avg_vol10']).copy()
# df_full_test[f'r_av_v5_20'] = (df_full_test[f'avg_vol5'] / df_full_test[f'avg_vol20']).copy()
# df_full_test[f'r_av_v5_60'] = (df_full_test[f'avg_vol5'] / df_full_test[f'avg_vol60']).copy()
# df_full_test[f'r_av_v10_20'] = (df_full_test[f'avg_vol10'] / df_full_test[f'avg_vol20']).copy()
# df_full_test[f'r_av_v10_60'] = (df_full_test[f'avg_vol10'] / df_full_test[f'avg_vol60']).copy()
# df_full_test[f'r_av_v10_120'] = (df_full_test[f'avg_vol10'] / df_full_test[f'avg_vol120']).copy()
# df_full_test[f'r_av_v20_60'] = (df_full_test[f'avg_vol20'] / df_full_test[f'avg_vol60']).copy()
# df_full_test[f'r_av_v20_120'] = (df_full_test[f'avg_vol20'] / df_full_test[f'avg_vol120']).copy()
# df_full_test[f'r_av_v60_120'] = (df_full_test[f'avg_vol60'] / df_full_test[f'avg_vol120']).copy()

# df_full_l20d[f'r_av_v5_10'] = (df_full_l20d[f'avg_vol5'] / df_full_l20d[f'avg_vol10']).copy()
# df_full_l20d[f'r_av_v5_20'] = (df_full_l20d[f'avg_vol5'] / df_full_l20d[f'avg_vol20']).copy()
# df_full_l20d[f'r_av_v5_60'] = (df_full_l20d[f'avg_vol5'] / df_full_l20d[f'avg_vol60']).copy()
# df_full_l20d[f'r_av_v10_20'] = (df_full_l20d[f'avg_vol10'] / df_full_l20d[f'avg_vol20']).copy()
# df_full_l20d[f'r_av_v10_60'] = (df_full_l20d[f'avg_vol10'] / df_full_l20d[f'avg_vol60']).copy()
# df_full_l20d[f'r_av_v10_120'] = (df_full_l20d[f'avg_vol10'] / df_full_l20d[f'avg_vol120']).copy()
# df_full_l20d[f'r_av_v20_60'] = (df_full_l20d[f'avg_vol20'] / df_full_l20d[f'avg_vol60']).copy()
# df_full_l20d[f'r_av_v20_120'] = (df_full_l20d[f'avg_vol20'] / df_full_l20d[f'avg_vol120']).copy()
# df_full_l20d[f'r_av_v60_120'] = (df_full_l20d[f'avg_vol60'] / df_full_l20d[f'avg_vol120']).copy()

In [16]:
# for icol in df_full_l20d.columns:
#     print(f"'{icol}',")

## ML

In [17]:
col_feats = ['RSI_3',
'RSI_5',
'RSI_10',
'RSI_15',
'RSI_20',
'RSI_60',
'RSI_120',
'ADX_3',
'DMP_3',
'DMN_3',
'ADX_5',
'DMP_5',
'DMN_5',
'ADX_10',
'DMP_10',
'DMN_10',
'ADX_15',
'DMP_15',
'DMN_15',
'ADX_20',
'DMP_20',
'DMN_20',
'ADX_60',
'DMP_60',
'DMN_60',
'ADX_120',
'DMP_120',
'DMN_120',
'OBV',
'STOCHk_14_3_3',
'STOCHd_14_3_3',
'ISA_9',
'ISB_26',
'ITS_9',
'IKS_26',
'MACD_5_10_9',
'MACDh_5_10_9',
'MACDs_5_10_9',
'diff_MACD_5_10_9',
'diff_MACDh_5_10_9',
'diff_MACDs_5_10_9',
'diff_RSI_3',
'diff_EMA_3',
'diff_BBL_3',
'diff_BBM_3',
'diff_BBU_3',
'diff_BBB_3',
'diff_BBP_3',
'r_close_bbl_3',
'r_close_bbu_3',
'r_close_bbm_3',
'r_close_bbb_3',
'r_close_bbp_3',
'r_sma3_bbl_3',
'r_sma3_bbu_3',
'r_sma3_bbm_3',
'r_sma3_bbb_3',
'r_sma3_bbp_3',
'r_sma5_bbl_3',
'r_sma5_bbu_3',
'r_sma5_bbm_3',
'r_sma5_bbb_3',
'r_sma5_bbp_3',
'diff_RSI_5',
'diff_EMA_5',
'diff_BBL_5',
'diff_BBM_5',
'diff_BBU_5',
'diff_BBB_5',
'diff_BBP_5',
'r_close_bbl_5',
'r_close_bbu_5',
'r_close_bbm_5',
'r_close_bbb_5',
'r_close_bbp_5',
'r_sma3_bbl_5',
'r_sma3_bbu_5',
'r_sma3_bbm_5',
'r_sma3_bbb_5',
'r_sma3_bbp_5',
'r_sma5_bbl_5',
'r_sma5_bbu_5',
'r_sma5_bbm_5',
'r_sma5_bbb_5',
'r_sma5_bbp_5',
'diff_RSI_10',
'diff_EMA_10',
'diff_BBL_10',
'diff_BBM_10',
'diff_BBU_10',
'diff_BBB_10',
'diff_BBP_10',
'r_close_bbl_10',
'r_close_bbu_10',
'r_close_bbm_10',
'r_close_bbb_10',
'r_close_bbp_10',
'r_sma3_bbl_10',
'r_sma3_bbu_10',
'r_sma3_bbm_10',
'r_sma3_bbb_10',
'r_sma3_bbp_10',
'r_sma5_bbl_10',
'r_sma5_bbu_10',
'r_sma5_bbm_10',
'r_sma5_bbb_10',
'r_sma5_bbp_10',
'diff_RSI_15',
'diff_EMA_15',
'diff_BBL_15',
'diff_BBM_15',
'diff_BBU_15',
'diff_BBB_15',
'diff_BBP_15',
'r_close_bbl_15',
'r_close_bbu_15',
'r_close_bbm_15',
'r_close_bbb_15',
'r_close_bbp_15',
'r_sma3_bbl_15',
'r_sma3_bbu_15',
'r_sma3_bbm_15',
'r_sma3_bbb_15',
'r_sma3_bbp_15',
'r_sma5_bbl_15',
'r_sma5_bbu_15',
'r_sma5_bbm_15',
'r_sma5_bbb_15',
'r_sma5_bbp_15',
'diff_RSI_20',
'diff_EMA_20',
'diff_BBL_20',
'diff_BBM_20',
'diff_BBU_20',
'diff_BBB_20',
'diff_BBP_20',
'r_close_bbl_20',
'r_close_bbu_20',
'r_close_bbm_20',
'r_close_bbb_20',
'r_close_bbp_20',
'r_sma3_bbl_20',
'r_sma3_bbu_20',
'r_sma3_bbm_20',
'r_sma3_bbb_20',
'r_sma3_bbp_20',
'r_sma5_bbl_20',
'r_sma5_bbu_20',
'r_sma5_bbm_20',
'r_sma5_bbb_20',
'r_sma5_bbp_20',
'diff_RSI_60',
'diff_EMA_60',
'diff_BBL_60',
'diff_BBM_60',
'diff_BBU_60',
'diff_BBB_60',
'diff_BBP_60',
'r_close_bbl_60',
'r_close_bbu_60',
'r_close_bbm_60',
'r_close_bbb_60',
'r_close_bbp_60',
'r_sma3_bbl_60',
'r_sma3_bbu_60',
'r_sma3_bbm_60',
'r_sma3_bbb_60',
'r_sma3_bbp_60',
'r_sma5_bbl_60',
'r_sma5_bbu_60',
'r_sma5_bbm_60',
'r_sma5_bbb_60',
'r_sma5_bbp_60',
'diff_RSI_120',
'diff_EMA_120',
'diff_BBL_120',
'diff_BBM_120',
'diff_BBU_120',
'diff_BBB_120',
'diff_BBP_120',
'r_close_bbl_120',
'r_close_bbu_120',
'r_close_bbm_120',
'r_close_bbb_120',
'r_close_bbp_120',
'r_sma3_bbl_120',
'r_sma3_bbu_120',
'r_sma3_bbm_120',
'r_sma3_bbb_120',
'r_sma3_bbp_120',
'r_sma5_bbl_120',
'r_sma5_bbu_120',
'r_sma5_bbm_120',
'r_sma5_bbb_120',
'r_sma5_bbp_120',
'r_rsi_3_5',
'r_rsi_3_10',
'r_rsi_5_10',
'r_rsi_5_15',
'r_rsi_10_15',
'r_rsi_10_20',
'r_rsi_15_20',
'r_rsi_15_60',
'r_rsi_20_60',
'r_rsi_20_120',
'r_rsi_60_120',
'r_ema_close_3',
'r_ema_close_5',
'r_ema_3_5',
'r_ema_3_10',
'r_ema_5_10',
'r_ema_5_15',
'r_ema_10_15',
'r_ema_10_20',
'r_ema_15_20',
'r_ema_15_60',
'r_ema_20_60',
'r_ema_20_120',
'r_ema_60_120',
'r_stoch_k_d',
'up5',
'down5',
'up10',
'down10',
'up20',
'down20',
'up60',
'down60',
'up120',
'down120',
'r_up_down5',
'r_up_down10',
'r_up_down20',
'r_up_down60',
'r_up_down120',
'r_curr_vol_5',
'r_curr_vol_10',
'r_curr_vol_20',
'r_curr_vol_60',
'r_curr_vol_120',
'intraday_direction',
'perc_max_spread_to_close',
'perc_max_spread_to_open',
'grad_MACD_5_10_9',
'grad_MACDh_5_10_9',
'grad_MACDs_5_10_9',
'grad_RSI_3',
'grad_EMA_3',
'grad_BBL_3',
'grad_BBM_3',
'grad_BBU_3',
'grad_BBB_3',
'grad_BBP_3',
'r_close_rsi_3',
'r_open_rsi_3',
'r_low_rsi_3',
'r_high_rsi_3',
'r_close_EMA_3',
'r_open_EMA_3',
'r_low_EMA_3',
'r_high_EMA_3',
'r_close_SMA_3',
'r_open_SMA_3',
'r_low_SMA_3',
'r_high_SMA_3',
'r_open_bbl_3',
'r_open_bbu_3',
'r_open_bbm_3',
'r_open_bbb_3',
'r_open_bbp_3',
'r_low_bbl_3',
'r_low_bbu_3',
'r_low_bbm_3',
'r_low_bbb_3',
'r_low_bbp_3',
'r_high_bbl_3',
'r_high_bbu_3',
'r_high_bbm_3',
'r_high_bbb_3',
'r_high_bbp_3',
'grad_RSI_5',
'grad_EMA_5',
'grad_BBL_5',
'grad_BBM_5',
'grad_BBU_5',
'grad_BBB_5',
'grad_BBP_5',
'r_close_rsi_5',
'r_open_rsi_5',
'r_low_rsi_5',
'r_high_rsi_5',
'r_close_EMA_5',
'r_open_EMA_5',
'r_low_EMA_5',
'r_high_EMA_5',
'r_close_SMA_5',
'r_open_SMA_5',
'r_low_SMA_5',
'r_high_SMA_5',
'r_open_bbl_5',
'r_open_bbu_5',
'r_open_bbm_5',
'r_open_bbb_5',
'r_open_bbp_5',
'r_low_bbl_5',
'r_low_bbu_5',
'r_low_bbm_5',
'r_low_bbb_5',
'r_low_bbp_5',
'r_high_bbl_5',
'r_high_bbu_5',
'r_high_bbm_5',
'r_high_bbb_5',
'r_high_bbp_5',
'grad_RSI_10',
'grad_EMA_10',
'grad_BBL_10',
'grad_BBM_10',
'grad_BBU_10',
'grad_BBB_10',
'grad_BBP_10',
'r_close_rsi_10',
'r_open_rsi_10',
'r_low_rsi_10',
'r_high_rsi_10',
'r_close_EMA_10',
'r_open_EMA_10',
'r_low_EMA_10',
'r_high_EMA_10',
'r_close_SMA_10',
'r_open_SMA_10',
'r_low_SMA_10',
'r_high_SMA_10',
'r_open_bbl_10',
'r_open_bbu_10',
'r_open_bbm_10',
'r_open_bbb_10',
'r_open_bbp_10',
'r_low_bbl_10',
'r_low_bbu_10',
'r_low_bbm_10',
'r_low_bbb_10',
'r_low_bbp_10',
'r_high_bbl_10',
'r_high_bbu_10',
'r_high_bbm_10',
'r_high_bbb_10',
'r_high_bbp_10',
'grad_RSI_15',
'grad_EMA_15',
'grad_BBL_15',
'grad_BBM_15',
'grad_BBU_15',
'grad_BBB_15',
'grad_BBP_15',
'r_close_rsi_15',
'r_open_rsi_15',
'r_low_rsi_15',
'r_high_rsi_15',
'r_close_EMA_15',
'r_open_EMA_15',
'r_low_EMA_15',
'r_high_EMA_15',
'r_close_SMA_15',
'r_open_SMA_15',
'r_low_SMA_15',
'r_high_SMA_15',
'r_open_bbl_15',
'r_open_bbu_15',
'r_open_bbm_15',
'r_open_bbb_15',
'r_open_bbp_15',
'r_low_bbl_15',
'r_low_bbu_15',
'r_low_bbm_15',
'r_low_bbb_15',
'r_low_bbp_15',
'r_high_bbl_15',
'r_high_bbu_15',
'r_high_bbm_15',
'r_high_bbb_15',
'r_high_bbp_15',
'grad_RSI_20',
'grad_EMA_20',
'grad_BBL_20',
'grad_BBM_20',
'grad_BBU_20',
'grad_BBB_20',
'grad_BBP_20',
'r_close_rsi_20',
'r_open_rsi_20',
'r_low_rsi_20',
'r_high_rsi_20',
'r_close_EMA_20',
'r_open_EMA_20',
'r_low_EMA_20',
'r_high_EMA_20',
'r_close_SMA_20',
'r_open_SMA_20',
'r_low_SMA_20',
'r_high_SMA_20',
'r_open_bbl_20',
'r_open_bbu_20',
'r_open_bbm_20',
'r_open_bbb_20',
'r_open_bbp_20',
'r_low_bbl_20',
'r_low_bbu_20',
'r_low_bbm_20',
'r_low_bbb_20',
'r_low_bbp_20',
'r_high_bbl_20',
'r_high_bbu_20',
'r_high_bbm_20',
'r_high_bbb_20',
'r_high_bbp_20',
'grad_RSI_60',
'grad_EMA_60',
'grad_BBL_60',
'grad_BBM_60',
'grad_BBU_60',
'grad_BBB_60',
'grad_BBP_60',
'r_close_rsi_60',
'r_open_rsi_60',
'r_low_rsi_60',
'r_high_rsi_60',
'r_close_EMA_60',
'r_open_EMA_60',
'r_low_EMA_60',
'r_high_EMA_60',
'r_close_SMA_60',
'r_open_SMA_60',
'r_low_SMA_60',
'r_high_SMA_60',
'r_open_bbl_60',
'r_open_bbu_60',
'r_open_bbm_60',
'r_open_bbb_60',
'r_open_bbp_60',
'r_low_bbl_60',
'r_low_bbu_60',
'r_low_bbm_60',
'r_low_bbb_60',
'r_low_bbp_60',
'r_high_bbl_60',
'r_high_bbu_60',
'r_high_bbm_60',
'r_high_bbb_60',
'r_high_bbp_60',
'grad_RSI_120',
'grad_EMA_120',
'grad_BBL_120',
'grad_BBM_120',
'grad_BBU_120',
'grad_BBB_120',
'grad_BBP_120',
'r_close_rsi_120',
'r_open_rsi_120',
'r_low_rsi_120',
'r_high_rsi_120',
'r_close_EMA_120',
'r_open_EMA_120',
'r_low_EMA_120',
'r_high_EMA_120',
'r_close_SMA_120',
'r_open_SMA_120',
'r_low_SMA_120',
'r_high_SMA_120',
'r_open_bbl_120',
'r_open_bbu_120',
'r_open_bbm_120',
'r_open_bbb_120',
'r_open_bbp_120',
'r_low_bbl_120',
'r_low_bbu_120',
'r_low_bbm_120',
'r_low_bbb_120',
'r_low_bbp_120',
'r_high_bbl_120',
'r_high_bbu_120',
'r_high_bbm_120',
'r_high_bbb_120',
'r_high_bbp_120',
'r_dmp_adx_3',
'r_dmn_adx_3',
'r_dmp_adx_5',
'r_dmn_adx_5',
'r_dmp_adx_10',
'r_dmn_adx_10',
'r_dmp_adx_15',
'r_dmn_adx_15',
'r_dmp_adx_20',
'r_dmn_adx_20',
'r_dmp_adx_60',
'r_dmn_adx_60',
'r_dmp_adx_120',
'r_dmn_adx_120',
'r_av_v5_10',
'r_av_v5_20',
'r_av_v5_60',
'r_av_v10_20',
'r_av_v10_60',
'r_av_v10_120',
'r_av_v20_60',
'r_av_v20_120',
'r_av_v60_120']

In [18]:
col_labels = ['label_close2max',
'label_close3max',
'label_close5max',
'label_close10max',
'label_close15max',
'label_close20max',
'label_close2min',
'label_close3min',
'label_close5min',
'label_close10min',
'label_close15min',
'label_close20min',
'label_close2mean',
'label_close3mean',
'label_close5mean',
'label_close10mean',
'label_close15mean',
'label_close20mean',
'inc_close2mean_perc5',
'inc_close2mean_perc10',
'inc_close2mean_perc3',
'inc_close2max_perc5',
'inc_close2max_perc10',
'inc_close2max_perc3',
'inc_close3mean_perc5',
'inc_close3mean_perc10',
'inc_close3mean_perc3',
'inc_close3max_perc5',
'inc_close3max_perc10',
'inc_close3max_perc3',
'inc_close5mean_perc5',
'inc_close5mean_perc10',
'inc_close5mean_perc3',
'inc_close5max_perc5',
'inc_close5max_perc10',
'inc_close5max_perc3',
'inc_close10mean_perc5',
'inc_close10mean_perc10',
'inc_close10mean_perc3',
'inc_close10max_perc5',
'inc_close10max_perc10',
'inc_close10max_perc3',
'dec_close2mean_perc5',
'dec_close2mean_perc10',
'dec_close2mean_perc3',
'dec_close2min_perc5',
'dec_close2min_perc10',
'dec_close2min_perc3',
'dec_close3mean_perc5',
'dec_close3mean_perc10',
'dec_close3mean_perc3',
'dec_close3min_perc5',
'dec_close3min_perc10',
'dec_close3min_perc3',
'dec_close5mean_perc5',
'dec_close5mean_perc10',
'dec_close5mean_perc3',
'dec_close5min_perc5',
'dec_close5min_perc10',
'dec_close5min_perc3',
'dec_close10mean_perc5',
'dec_close10mean_perc10',
'dec_close10mean_perc3',
'dec_close10min_perc5',
'dec_close10min_perc10',
'dec_close10min_perc3']

### Correlation

In [19]:
corr_mat = df_full_train[col_feats].corr()

In [20]:
corr_mat.head()

,RSI_3,RSI_5,RSI_10,RSI_15,RSI_20,RSI_60,RSI_120,ADX_3,DMP_3,DMN_3,ADX_5,DMP_5,DMN_5,ADX_10,DMP_10,DMN_10,ADX_15,DMP_15,DMN_15,ADX_20,DMP_20,DMN_20,ADX_60,DMP_60,DMN_60,ADX_120,DMP_120,DMN_120,OBV,STOCHk_14_3_3,STOCHd_14_3_3,ISA_9,ISB_26,ITS_9,IKS_26,MACD_5_10_9,MACDh_5_10_9,MACDs_5_10_9,diff_MACD_5_10_9,diff_MACDh_5_10_9,diff_MACDs_5_10_9,diff_RSI_3,diff_EMA_3,diff_BBL_3,diff_BBM_3,diff_BBU_3,diff_BBB_3,diff_BBP_3,r_close_bbl_3,r_close_bbu_3,r_close_bbm_3,r_close_bbb_3,r_close_bbp_3,r_sma3_bbl_3,r_sma3_bbu_3,r_sma3_bbm_3,r_sma3_bbb_3,r_sma3_bbp_3,r_sma5_bbl_3,r_sma5_bbu_3,r_sma5_bbm_3,r_sma5_bbb_3,r_sma5_bbp_3,diff_RSI_5,diff_EMA_5,diff_BBL_5,diff_BBM_5,diff_BBU_5,diff_BBB_5,diff_BBP_5,r_close_bbl_5,r_close_bbu_5,r_close_bbm_5,r_close_bbb_5,r_close_bbp_5,r_sma3_bbl_5,r_sma3_bbu_5,r_sma3_bbm_5,r_sma3_bbb_5,r_sma3_bbp_5,r_sma5_bbl_5,r_sma5_bbu_5,r_sma5_bbm_5,r_sma5_bbb_5,r_sma5_bbp_5,diff_RSI_10,diff_EMA_10,diff_BBL_10,diff_BBM_10,diff_BBU_10,diff_BBB_10,diff_BBP_10,r_close_bbl_10,r_close_bbu_10,r_close_bbm_10,r_close_bbb_10,r_close_bbp_10,r_sma3_bbl_10,r_sma3_bbu_10,r_sma3_bbm_10,r_sma3_bbb_10,r_sma3_bbp_10,r_sma5_bbl_10,r_sma5_bbu_10,r_sma5_bbm_10,r_sma5_bbb_10,r_sma5_bbp_10,diff_RSI_15,diff_EMA_15,diff_BBL_15,diff_BBM_15,diff_BBU_15,diff_BBB_15,diff_BBP_15,r_close_bbl_15,r_close_bbu_15,r_close_bbm_15,r_close_bbb_15,r_close_bbp_15,r_sma3_bbl_15,r_sma3_bbu_15,r_sma3_bbm_15,r_sma3_bbb_15,r_sma3_bbp_15,r_sma5_bbl_15,r_sma5_bbu_15,r_sma5_bbm_15,r_sma5_bbb_15,r_sma5_bbp_15,diff_RSI_20,diff_EMA_20,diff_BBL_20,diff_BBM_20,diff_BBU_20,diff_BBB_20,diff_BBP_20,r_close_bbl_20,r_close_bbu_20,r_close_bbm_20,r_close_bbb_20,r_close_bbp_20,r_sma3_bbl_20,r_sma3_bbu_20,r_sma3_bbm_20,r_sma3_bbb_20,r_sma3_bbp_20,r_sma5_bbl_20,r_sma5_bbu_20,r_sma5_bbm_20,r_sma5_bbb_20,r_sma5_bbp_20,diff_RSI_60,diff_EMA_60,diff_BBL_60,diff_BBM_60,diff_BBU_60,diff_BBB_60,diff_BBP_60,r_close_bbl_60,r_close_bbu_60,r_close_bbm_60,r_close_bbb_60,r_close_bbp_60,r_sma3_bbl_60,r_sma3_bbu_60,r_sma3_bbm_60,r_sma3_bbb_60,r_sma3_bbp_60,r_sma5_bbl_60,r_sma5_bbu_60,r_sma5_bbm_60,r_sma5_bbb_60,r_sma5_bbp_60,diff_RSI_120,diff_EMA_120,diff_BBL_120,diff_BBM_120,diff_BBU_120,diff_BBB_120,diff_BBP_120,r_close_bbl_120,r_close_bbu_120,r_close_bbm_120,r_close_bbb_120,r_close_bbp_120,r_sma3_bbl_120,r_sma3_bbu_120,r_sma3_bbm_120,r_sma3_bbb_120,r_sma3_bbp_120,r_sma5_bbl_120,r_sma5_bbu_120,r_sma5_bbm_120,r_sma5_bbb_120,r_sma5_bbp_120,r_rsi_3_5,r_rsi_3_10,r_rsi_5_10,r_rsi_5_15,r_rsi_10_15,r_rsi_10_20,r_rsi_15_20,r_rsi_15_60,r_rsi_20_60,r_rsi_20_120,r_rsi_60_120,r_ema_close_3,r_ema_close_5,r_ema_3_5,r_ema_3_10,r_ema_5_10,r_ema_5_15,r_ema_10_15,r_ema_10_20,r_ema_15_20,r_ema_15_60,r_ema_20_60,r_ema_20_120,r_ema_60_120,r_stoch_k_d,up5,down5,up10,down10,up20,down20,up60,down60,up120,down120,r_up_down5,r_up_down10,r_up_down20,r_up_down60,r_up_down120,r_curr_vol_5,r_curr_vol_10,r_curr_vol_20,r_curr_vol_60,r_curr_vol_120,intraday_direction,perc_max_spread_to_close,perc_max_spread_to_open,grad_MACD_5_10_9,grad_MACDh_5_10_9,grad_MACDs_5_10_9,grad_RSI_3,grad_EMA_3,grad_BBL_3,grad_BBM_3,grad_BBU_3,grad_BBB_3,grad_BBP_3,r_close_rsi_3,r_open_rsi_3,r_low_rsi_3,r_high_rsi_3,r_close_EMA_3,r_open_EMA_3,r_low_EMA_3,r_high_EMA_3,r_close_SMA_3,r_open_SMA_3,r_low_SMA_3,r_high_SMA_3,r_open_bbl_3,r_open_bbu_3,r_open_bbm_3,r_open_bbb_3,r_open_bbp_3,r_low_bbl_3,r_low_bbu_3,r_low_bbm_3,r_low_bbb_3,r_low_bbp_3,r_high_bbl_3,r_high_bbu_3,r_high_bbm_3,r_high_bbb_3,r_high_bbp_3,grad_RSI_5,grad_EMA_5,grad_BBL_5,grad_BBM_5,grad_BBU_5,grad_BBB_5,grad_BBP_5,r_close_rsi_5,r_open_rsi_5,r_low_rsi_5,r_high_rsi_5,r_close_EMA_5,r_open_EMA_5,r_low_EMA_5,r_high_EMA_5,r_close_SMA_5,r_open_SMA_5,r_low_SMA_5,r_high_SMA_5,r_open_bbl_5,r_open_bbu_5,r_open_bbm_5,r_open_bbb_5,r_open_bbp_5,r_low_bbl_5,r_low_bbu_5,r_low_bbm_5,r_low_bbb_5,r_low_bbp_5,r_high_bbl_5,r_high_bbu_5,r_high_bbm_5,r_high_bbb_5,r_high_bbp_5,grad_RSI_10,grad_EMA_10,grad_BBL_10,grad_BBM_10,grad_BBU_10,grad_BBB_10,grad_BBP_10,r_close_rsi_10,r_open_rsi_10,r_low_rsi_10,r_high

### Feature gini

In [21]:
col_target = 'inc_close5mean_perc10'

In [22]:
from sklearn.metrics import roc_auc_score
import math

feature_auc = pd.Series(index=col_feats)
mask = (df_full_train[col_target].isnull()==False)

for icol in col_feats:
    mask = (df_full_train[col_target].isnull()==False) & (df_full_train[icol].isnull()==False) & (np.isinf(df_full_train[icol])==False)
    feature_auc.loc[icol] = roc_auc_score(df_full_train.loc[mask, col_target], df_full_train.loc[mask, icol])

In [23]:
feature_auc.loc[feature_auc<0.5] = 1 - feature_auc

In [24]:
feature_auc.sort_values(ascending=False).to_clipboard()

### Train Valid

In [25]:
from sklearn.model_selection import train_test_split

train_idx, valid_idx, y_train, y_valid = train_test_split(df_full_train.index, df_full_train[col_target], test_size=0.4, random_state=42, stratify=df_full_train[col_target])

### Price increase

In [27]:
col_target = 'inc_close2max_perc5'

In [28]:
col_predict_proba = f"predict_proba_{col_target}"
col_predict = f"predict_{col_target}"

In [29]:
print(df_full_train[col_target].value_counts())
print(df_full_train[col_target].value_counts()/df_full_train[col_target].count())

inc_close2max_perc5
0    358586
1     44814
Name: count, dtype: int64
inc_close2max_perc5
0    0.888909
1    0.111091
Name: count, dtype: float64


In [30]:
# !pip install xgboost

In [31]:
import lightgbm as lgb
clf = lgb.LGBMClassifier(default='split',  min_data_in_leaf=300)
# clf = lgb.LGBMClassifier()

clf.fit(df_full_train.loc[train_idx, col_feats], df_full_train.loc[train_idx, col_target])

F:\ProgramData\miniconda3\envs\nlp\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "F:\ProgramData\miniconda3\envs\nlp\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


[LightGBM] [Warning] Unknown parameter: default
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] Unknown parameter: default
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Info] Number of positive: 26937, number of negative: 215103
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.517967 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 126185
[LightGBM] [Info] Number of data points in the train set: 242040, number of used features: 506
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.111292 -> initscore=-2.077616
[LightGBM] [Info] Start training from score -2.077616


LGBMClassifier(default='split', min_data_in_leaf=300)

In [32]:
df_full_test[col_predict] = clf.predict(df_full_test[col_feats])
df_full_test[col_predict_proba] = clf.predict_proba(df_full_test[col_feats])[:,1]

[LightGBM] [Warning] Unknown parameter: default
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] Unknown parameter: default
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300


In [33]:
from sklearn.metrics import roc_auc_score

print(roc_auc_score(df_full_test[col_target], df_full_test[col_predict_proba]))

df_full_test['cut_proba'] = np.nan
df_full_test['cut_proba'] = pd.cut(df_full_test[col_predict_proba], 10, labels=False)
display(df_full_test[['cut_proba', col_predict_proba, col_target]].groupby('cut_proba').agg(['mean', 'count']))

0.8113627581014616


predict_proba_inc_close2max_perc5         inc_close2max_perc5  \
                                       mean   count                mean   
cut_proba                                                                 
0                                  0.028429  233935            0.025661   
1                                  0.119480   69007            0.120596   
2                                  0.201690   42856            0.206599   
3                                  0.281285   24176            0.284662   
4                                  0.360794    9651            0.341519   
5                                  0.441377    3263            0.412197   
6                                  0.521806     875            0.499429   
7                                  0.603142     192            0.598958   
8                                  0.686300      26            0.576923   
9                                  0.777701       2            0.500000   

                   
            count  
cut_proba          
0          233935  
1           69007  
2           42856  
3           24176  
4            9651  
5            3263  
6             875  
7             192  
8              26  
9               2

In [ ]:
explainer = shap.TreeExplainer(clf)
shap_values = explainer(df_full_test[col_feats])
importance_sr = pd.Series(np.abs(shap_values.values).mean(axis=0), index=col_feats, name='importance')
importance_sr.sort_values(ascending=False, inplace=True)
use_feat = importance_sr.index.tolist()

In [ ]:
# use_feat = pd.Series(clf.feature_importances_, index=col_feats).sort_values(ascending=False).index.tolist()
# print(importance_sr)

In [ ]:
def corr_gain(list_feat, corr_mat, list_use, th_corr=0.3):
    if len(list_feat)>0:
        mat_corr = corr_mat.loc[list_feat[0], list_feat[1:]].abs()
        del_mat = mat_corr.loc[mat_corr>th_corr].index.to_list()
        list_feat = [ifeat for ifeat in list_feat if ifeat not in del_mat]
        list_use.append(list_feat[0])
        list_feat = list_feat[1:]
        corr_gain(list_feat, corr_mat, list_use)
    else:
        return None

In [ ]:
list_use = list()

corr_gain(use_feat, corr_mat, list_use)

#### Train with selected features

In [ ]:
len(list_use)

In [ ]:
import lightgbm as lgb
clf_inc = lgb.LGBMClassifier(default='split', min_data_in_leaf=300)

clf_inc.fit(df_full_train[list_use], df_full_train[col_target])

In [ ]:
df_full_test[col_predict] = clf_inc.predict(df_full_test[list_use])
df_full_test[col_predict_proba] = clf_inc.predict_proba(df_full_test[list_use])[:,1]
df_full_l20d[col_predict] = clf_inc.predict(df_full_l20d[list_use])
df_full_l20d[col_predict_proba] = clf_inc.predict_proba(df_full_l20d[list_use])[:,1]

In [ ]:
from sklearn.metrics import roc_auc_score

print(roc_auc_score(df_full_test[col_target], df_full_test[col_predict_proba]))

df_full_test['cut_proba'] = np.nan
df_full_test['cut_proba'] = pd.cut(df_full_test[col_predict_proba], 20, labels=False)
display(df_full_test[['cut_proba', col_predict_proba, col_target]].groupby('cut_proba').agg(['mean', 'count']))

In [ ]:
explainer = shap.TreeExplainer(clf_inc)
shap_values = explainer(df_full_test[list_use])
shap.summary_plot(shap_values, df_full_test[list_use])

In [ ]:
use_feat = pd.Series(clf_inc.feature_importances_, index=list_use).sort_values(ascending=False)
# list_use = use_feat[use_feat>20].index

In [ ]:
use_feat

In [ ]:
list_use_final = use_feat[:10].index.tolist()

In [ ]:
list_use_final

In [ ]:
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

def objective(trial):
    learning_r = trial.suggest_float('learning_rate', 0.01, 0.5, step=0.01)
    n_data_leaf = trial.suggest_int("min_data_in_leaf", 50, 1000, step = 10)
    n_depth = trial.suggest_int("max_depth", 3, 10, step = 1)
    clf_inc = lgb.LGBMClassifier(default='split', min_data_in_leaf=n_data_leaf, max_depth=n_depth)
    clf_inc.fit(df_full_train.loc[train_idx, list_use_final], df_full_train.loc[train_idx, col_target])
    preds = clf_inc.predict_proba(df_full_train.loc[valid_idx, list_use_final])[:,1]
    pred_labels = np.rint(df_full_train.loc[valid_idx, col_target])
    return roc_auc_score(pred_labels,preds )

# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

In [ ]:
# Print the best set of hyperparameters
print('Best hyperparameters: ', study.best_params)
# Print the corresponding performance
print('Best performance: ', study.best_value)

In [ ]:
import lightgbm as lgb
clf_inc = lgb.LGBMClassifier(default='split', 
                             min_data_in_leaf=study.best_params['min_data_in_leaf'],
                            learning_rate=study.best_params['learning_rate'],
                            max_depth=study.best_params['max_depth'])

clf_inc.fit(df_full_train[list_use_final], df_full_train[col_target])

In [ ]:
df_full_test[col_predict] = clf_inc.predict(df_full_test[list_use_final])
df_full_test[col_predict_proba] = clf_inc.predict_proba(df_full_test[list_use_final])[:,1]
df_full_l20d[col_predict] = clf_inc.predict(df_full_l20d[list_use_final])
df_full_l20d[col_predict_proba] = clf_inc.predict_proba(df_full_l20d[list_use_final])[:,1]

In [ ]:
from sklearn.metrics import roc_auc_score

print(roc_auc_score(df_full_test[col_target], df_full_test[col_predict_proba]))

df_full_test['cut_proba'] = np.nan
df_full_test['cut_proba'] = pd.cut(df_full_test[col_predict_proba], 10, labels=False)
display(df_full_test[['cut_proba', col_predict_proba, col_target]].groupby('cut_proba').agg(['mean', 'count']))

In [ ]:
explainer = shap.TreeExplainer(clf_inc)
shap_values = explainer(df_full_test[list_use_final])
shap.summary_plot(shap_values, df_full_test[list_use_final])

In [ ]:
df_full_l20d.head(-20)

### Price Drop

In [ ]:
col_target = 'dec_close3min_perc5'
col_predict_proba = f"predict_proba_{col_target}"
col_predict = f"predict_{col_target}"

In [ ]:
print(df_full_train[col_target].value_counts())
print(df_full_train[col_target].value_counts()/df_full_train[col_target].count())

In [ ]:
import lightgbm as lgb
clf = lgb.LGBMClassifier(default='split', min_data_in_leaf=300)

clf.fit(df_full_train[col_feats], df_full_train[col_target])

In [ ]:
df_full_test[col_predict] = clf.predict(df_full_test[col_feats])
df_full_test[col_predict_proba] = clf.predict_proba(df_full_test[col_feats])[:,1]

In [ ]:
from sklearn.metrics import roc_auc_score

print(roc_auc_score(df_full_test[col_target], df_full_test[col_predict_proba]))

df_full_test['cut_proba'] = np.nan
df_full_test['cut_proba'] = pd.cut(df_full_test[col_predict_proba], 10, labels=False)
df_full_test[['cut_proba', col_predict_proba, col_target]].groupby('cut_proba').agg(['mean', 'count'])

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(df_full_test[col_target], df_full_test[col_predict])
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

In [ ]:
explainer = shap.TreeExplainer(clf)
shap_values = explainer(df_full_test[col_feats])
# shap.summary_plot(shap_values, df_full_test[col_feats])

In [ ]:
importance_sr = pd.Series(np.abs(shap_values.values).mean(axis=0), index=col_feats, name='importance')
importance_sr.sort_values(ascending=False, inplace=True)
use_feat = importance_sr.index.tolist()

In [ ]:
def corr_gain(list_feat, corr_mat, list_use):
    if len(list_feat)>0:
        mat_corr = corr_mat.loc[list_feat[0], list_feat[1:]].abs()
        del_mat = mat_corr.loc[mat_corr>0.5].index.to_list()
        list_feat = [ifeat for ifeat in list_feat if ifeat not in del_mat]
        list_use.append(list_feat[0])
        list_feat = list_feat[1:]
        corr_gain(list_feat, corr_mat, list_use)
    else:
        return None

In [ ]:
list_use = list()
use_feat = use_feat

corr_gain(use_feat, corr_mat, list_use)

#### Train Using selected features

In [ ]:
import lightgbm as lgb
clf_drop = lgb.LGBMClassifier(default='split', min_data_in_leaf=300)

clf_drop.fit(df_full_train[list_use], df_full_train[col_target])

In [ ]:
df_full_test[col_predict] = clf_drop.predict(df_full_test[list_use])
df_full_test[col_predict_proba] = clf_drop.predict_proba(df_full_test[list_use])[:,1]
df_full_l20d[col_predict] = clf_drop.predict(df_full_l20d[list_use])
df_full_l20d[col_predict_proba] = clf_drop.predict_proba(df_full_l20d[list_use])[:,1]

In [ ]:
from sklearn.metrics import roc_auc_score

print(roc_auc_score(df_full_test[col_target], df_full_test[col_predict_proba]))

df_full_test['cut_proba'] = np.nan
df_full_test['cut_proba'] = pd.cut(df_full_test[col_predict_proba], 10, labels=False)
df_full_test[['cut_proba', col_predict_proba, col_target]].groupby('cut_proba').agg(['mean', 'count'])

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(df_full_test[col_target], df_full_test[col_predict])
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

In [ ]:
explainer = shap.TreeExplainer(clf_drop)
shap_values = explainer(df_full_test[list_use])
shap.summary_plot(shap_values, df_full_test[list_use])

## Result

In [ ]:
from datetime import date
date_select = date(2024, 11, 18)

In [ ]:
# df_full_l20d

In [ ]:
# df_full_l20d.dtypes.loc['date']

In [ ]:
df_full_l20d.loc[df_full_l20d.date.dt.strftime("%Y%m%d").astype(int)>=20241018, ['ticker','open', 'close', 'high', 'low', 'predict_proba_inc_close3max_perc10']].sort_values('predict_proba_inc_close3max_perc10', ascending=False).head(20)

In [ ]:
df_full_l20d.loc[df_full_l20d.date.dt.strftime("%Y%m%d").astype(int)>=20241018, ['ticker','open', 'close', 'high', 'low', 'predict_proba_dec_close3min_perc5']].sort_values('predict_proba_dec_close3min_perc5', ascending=False).head(20)